In [ ]:
import requests

url = "https://api.linnworks.net/api/Auth/AuthorizeByApplication"

payload = {
    "ApplicationId": "ApplicationId",
    "ApplicationSecret": "ApplicationSecret",
    "Token": "Token"
}
headers = {
    "accept": "application/json",
    "content-type": "application/json"
}

response = requests.post(url, json=payload, headers=headers)

# Parse JSON response
data = response.json()

# Assuming the token is inside a key named 'Token' or 'AccessToken' or similar:
YOUR_ACCESS_TOKEN = data.get("Token")

print(YOUR_ACCESS_TOKEN)


In [ ]:
import requests
import pandas as pd
from datetime import datetime

#url="https://eu-ext.linnworks.net/api/PurchaseOrder/Get_PurchaseOrder"
url = "https://eu-ext.linnworks.net/api/PurchaseOrder/Search_PurchaseOrders2"

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": access_token  
}

payload = {
    "searchParameters": {
        "DateFrom": "2000-01-01T00:00:00",  # Very early to get all
        "DateTo": datetime.now().isoformat()
    },
    "entriesPerPage": 100,
    "pageNumber": 1
}

all_orders = []

while True:
    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        print(f"Error {response.status_code}: {response.text}")
        break

    data = response.json()
    orders = data.get("Result", [])
    all_orders.extend(orders)

    total_pages = data.get("TotalPages", 1)
    current_page = data.get("CurrentPageNumber", 1)

    print(f"Fetched page {current_page}/{total_pages}, {len(orders)} orders")

    if current_page >= total_pages:
        break

    payload["pageNumber"] += 1

# Convert to DataFrame with all fields
df = pd.DataFrame(all_orders)



Fetched page 1/4, 100 orders
Fetched page 2/4, 100 orders
Fetched page 3/4, 100 orders
Fetched page 4/4, 70 orders


In [ ]:

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "Authorization": access_token
}

entries_per_page = 100
page_number = 1
total_items = 0

while True:
    payload = {
        "keyword": "",
        "loadCompositeParents": True,
        "loadVariationParents": True,
        "entriesPerPage": entries_per_page,
        "pageNumber": page_number,
        "dataRequirements": ["StockLevels"],
        "searchTypes": ["SKU"]
    }

    response = requests.post(url, json=payload, headers=headers)
    if response.status_code != 200:
        raise RuntimeError(f"API error {response.status_code}: {response.text}")

    data = response.json()
    count = len(data)
    total_items += count

    if not data:  # no more items
        break

    page_number += 1

# Calculate total pages based on entries_per_page
total_pages = math.ceil(total_items / entries_per_page)
print(f"Total items fetched: {total_items}")
print(f"Estimated total pages: {total_pages}")


RuntimeError: API error 429: "API calls quota exceeded! Maximum admitted 150 per Minute."

In [29]:
df.count()

Suppliers                   200
StockLevels                 200
ItemChannelDescriptions     200
ItemExtendedProperties      200
ItemChannelTitles           200
ItemChannelPrices           200
Images                      200
ItemNumber                  200
ItemTitle                   200
BarcodeNumber               200
MetaData                    200
IsVariationParent           200
isBatchedStockType          200
PurchasePrice               200
TaxRate                     200
PostalServiceId             200
CategoryId                  200
CategoryName                200
PackageGroupId              200
Height                      200
Width                       200
Depth                       200
Weight                      200
CreationDate                200
InventoryTrackingType       200
BatchNumberScanRequired     200
SerialNumberScanRequired    200
StockItemId                 200
StockItemIntId              200
dtype: int64

In [31]:
df.columns

Index(['Suppliers', 'StockLevels', 'ItemChannelDescriptions',
       'ItemExtendedProperties', 'ItemChannelTitles', 'ItemChannelPrices',
       'Images', 'ItemNumber', 'ItemTitle', 'BarcodeNumber', 'MetaData',
       'IsVariationParent', 'isBatchedStockType', 'PurchasePrice', 'TaxRate',
       'PostalServiceId', 'CategoryId', 'CategoryName', 'PackageGroupId',
       'Height', 'Width', 'Depth', 'Weight', 'CreationDate',
       'InventoryTrackingType', 'BatchNumberScanRequired',
       'SerialNumberScanRequired', 'StockItemId', 'StockItemIntId'],
      dtype='object')

In [34]:
pd.set_option('display.max_colwidth', None)

In [38]:
df['StockLevels'].tail(1)

199    [{'Location': {'StockLocationId': '00000000-0000-0000-0000-000000000000', 'StockLocationIntId': 1, 'LocationName': 'Default', 'LocationTag': '', 'IsFulfillmentCenter': False, 'IsWarehouseManaged': False, 'BinRack': ''}, 'StockLevel': 0, 'StockValue': 0.0, 'MinimumLevel': 4, 'InOrderBook': 0, 'Due': 0, 'JIT': False, 'InOrders': 0, 'Available': 0, 'UnitCost': 0.0, 'AutoAdjust': False, 'LastUpdateDate': '2025-06-23T20:30:46.077Z', 'LastUpdateOperation': 'Imported from file', 'rowid': '00000000-0000-0000-0000-000000000000', 'PendingUpdate': False, 'StockItemPurchasePrice': 0.0, 'StockItemId': '653a39bb-9ac0-4581-9b11-8e10beee13d1', 'StockItemIntId': 0}, {'Location': {'StockLocationId': 'd3fc787a-596a-43a5-aaba-8baf9f3554fd', 'StockLocationIntId': 39, 'LocationName': 'Armorgard', 'LocationTag': '', 'IsFulfillmentCenter': True, 'IsWarehouseManaged': False, 'BinRack': ''}, 'StockLevel': 5, 'StockValue': 736.35, 'MinimumLevel': 4, 'InOrderBook': 0, 'Due': 0, 'JIT': False, 'InOrders': 0,